<a href="https://colab.research.google.com/github/HarshadaRaut234/BookRecSys/blob/main/BookRecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
books=pd.read_csv('/content/goodreads_data.csv.zip')

# New Section

In [4]:
books.head(1)

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...


In [5]:
books=books[['Book','Author','Description','Genres','Num_Ratings']]

In [6]:
books.head(2)

,Book,Author,Description,Genres,Num_Ratings
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...","5,691,311"
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...","9,278,135"


In [7]:
books.isnull().sum()

Book            0
Author          0
Description    77
Genres          0
Num_Ratings     0
dtype: int64

In [8]:
books.dropna(inplace=True)

In [9]:
books.duplicated().sum()

19

In [10]:
books.drop_duplicates(inplace=True)

To normalize num_ratings

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
sc=MinMaxScaler()

In [13]:
books['Num_Ratings'] = books['Num_Ratings'].str.replace(',', '').astype(float)


In [14]:
books['Num_Ratings']=sc.fit_transform(books[['Num_Ratings']])

In [15]:
books.head(1)

,Book,Author,Description,Genres,Num_Ratings
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",0.613411


Remove all the symbols from the title

In [16]:
import re
#books['Book']=re.sub(r'\([^()]*\)', '', books['Book'])
books['Book'] = books['Book'].str.replace(r'\([^)]*\)', '').str.strip()
# Assuming df is your DataFrame and 'Text_Column' is the column containing text with symbols
books['Book'] = books['Book'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
books['Description']=books['Description'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Display the DataFrame
print(books)


<ipython-input-16-9465e91bf7e7>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  books['Book'] = books['Book'].str.replace(r'\([^)]*\)', '').str.strip()


                                         Book             Author  \
0                       To Kill a Mockingbird         Harper Lee   
1     Harry Potter and the Philosophers Stone       J.K. Rowling   
2                         Pride and Prejudice        Jane Austen   
3                   The Diary of a Young Girl         Anne Frank   
4                                 Animal Farm      George Orwell   
...                                       ...                ...   
9995                                 Breeders     Ashley Quigley   
9996                                   Dynamo  Eleanor Gustafson   
9997                    The Republic of Trees         Sam Taylor   
9998                                Waking Up         Renee Dyer   
9999        Bits and Pieces Tales and Sonnets        Jas T. Ward   

                                            Description  \
0     The unforgettable novel of a childhood in a sl...   
1     Harry Potter thinks he is an ordinary boy  unt...   
2     

Creating new dataframe with combined features

In [17]:
Titles = books['Book']
combined_features = books['Description'] + ' ' + books['Author'] + ' ' + books['Genres'].apply(lambda x: ' '.join(x))

new_books = pd.DataFrame({'Book_Title': Titles, 'Combined_Features': combined_features})
print(new_books.head(10))

                                Book_Title  \
0                    To Kill a Mockingbird   
1  Harry Potter and the Philosophers Stone   
2                      Pride and Prejudice   
3                The Diary of a Young Girl   
4                              Animal Farm   
5                        The Little Prince   
6                                     1984   
7                         The Great Gatsby   
8                   The Catcher in the Rye   
9                    The Lord of the Rings   

                                   Combined_Features  
0  The unforgettable novel of a childhood in a sl...  
1  Harry Potter thinks he is an ordinary boy  unt...  
2  Since its immediate success in 1813 Pride and ...  
3  Discovered in the attic in which she spent the...  
4  Librarians note There is an Alternate Cover Ed...  
5  A pilot stranded in the desert awakes one morn...  
6  The new novel by George Orwell is the major wo...  
7  Alternate Cover Edition ISBN 0743273567 ISBN13... 

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)

# Fit and transform the combined features
combined_vectors = vectorizer.fit_transform(new_books['Combined_Features'])

# Convert the result to a DataFrame for easier inspection
combined_df = pd.DataFrame(combined_vectors.toarray(), columns=vectorizer.get_feature_names_out())

# Display the DataFrame with vectorized combined features
print(combined_df)

      ability      able  abuse  accident  acclaimed  account  achieve  act  \
0         0.0  0.000000    0.0       0.0        0.0  0.00000      0.0  0.0   
1         0.0  0.000000    0.0       0.0        0.0  0.00000      0.0  0.0   
2         0.0  0.000000    0.0       0.0        0.0  0.00000      0.0  0.0   
3         0.0  0.000000    0.0       0.0        0.0  0.13391      0.0  0.0   
4         0.0  0.000000    0.0       0.0        0.0  0.00000      0.0  0.0   
...       ...       ...    ...       ...        ...      ...      ...  ...   
9899      0.0  0.180121    0.0       0.0        0.0  0.00000      0.0  0.0   
9900      0.0  0.000000    0.0       0.0        0.0  0.00000      0.0  0.0   
9901      0.0  0.000000    0.0       0.0        0.0  0.00000      0.0  0.0   
9902      0.0  0.000000    0.0       0.0        0.0  0.00000      0.0  0.0   
9903      0.0  0.000000    0.0       0.0        0.0  0.00000      0.0  0.0   

      action  actually  ...  wrong  wrote  year     years  york

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

# Assuming description_df is the DataFrame with vectorized features
cosine_sim_matrix = cosine_similarity(combined_df)

# Display the cosine similarity matrix
print(cosine_sim_matrix)


[[1.         0.         0.03255774 ... 0.04994688 0.02980838 0.0446871 ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.03255774 0.         1.         ... 0.05742526 0.01595067 0.05551216]
 ...
 [0.04994688 0.         0.05742526 ... 1.         0.02001646 0.03646807]
 [0.02980838 0.         0.01595067 ... 0.02001646 1.         0.        ]
 [0.0446871  0.         0.05551216 ... 0.03646807 0.         1.        ]]


In [20]:
  def get_similar_books(book_title, cosine_sim_matrix, books, top_n=3):
      # Get the index of the target book
      book_index = new_books[new_books['Book_Title'] == book_title].index[0]

      # Get the cosine similarity scores for the target book
      cosine_scores = cosine_sim_matrix[book_index]

      # Get the indices of books most similar to the target book
      similar_books_indices = np.argsort(cosine_scores)[::-1][1:top_n+1]  # Exclude the target book itself

      # Get the titles of similar books
      similar_books = new_books.loc[similar_books_indices, 'Book_Title'].tolist()

      return similar_books

  # Example: Find top 3 similar books for a given book
  target_book_title = input()
  similar_books = get_similar_books(target_book_title, cosine_sim_matrix, books, top_n=3)

  # Display the result
  print(f"Books similar to '{target_book_title}': {similar_books}")

To Kill a Mockingbird
Books similar to 'To Kill a Mockingbird': ['The New Jim Crow Mass Incarceration in the Age of Colorblindness', 'De donkere kamer van Damokles', 'Wicked Fate']
